# XGBoost and Streamz

This notebook is a CPU comparison of the [Forest Inference Library](https://medium.com/rapids-ai/rapids-forest-inference-library-prediction-at-100-million-rows-per-second-19558890bc35) (FIL) and [cuStreamz](https://medium.com/rapids-ai/gpu-accelerated-stream-processing-with-rapids-f2b725696a61) pipeline using RAPIDS on the GPU.

View the GPU version of the [FIL custreamz pipeline](./FIL_custreamz_pipeline.ipynb)

## Data Download

In [ ]:
import s3fs
from os import path

# Download sample data and model
IOT_MALWARE_JSON="iot_malware_1_1.json"
IOT_XGBOOST_MODEL="iot_xgboost_model.bst"
S3_BASE_PATH = "rapidsai-data/cyber/clx"

In [ ]:
# xgboost model
if not path.exists(IOT_XGBOOST_MODEL):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + IOT_XGBOOST_MODEL, IOT_XGBOOST_MODEL)

In [ ]:
# IoT data in json format
if not path.exists(IOT_MALWARE_JSON):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + IOT_MALWARE_JSON, IOT_MALWARE_JSON)

With our kafka broker already running at `localhost:9092` and input kafka topic created, next we ingest our sample data into our topic named `input`. 

In [ ]:
# To load the data into kafka use the command line tool kafka-console-producer provided by your kafka installation. In this example kafka is installed at /opt/kafka.
# Update the broker-list and topic parameters as needed
!/opt/kafka/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic input < $IOT_MALWARE_JSON >/dev/null

# Optionally repeat this process to populate the kafka queue with more data.

# Imports

In [1]:
import random
import confluent_kafka as ck
import pandas as pd
import dask
from dask.distributed import Client, LocalCluster
from streamz import Stream
import time
import pandas as pd
import json
import xgboost as xgb

# Parameters

The average log size is used later in the notebook to estimate throughput and avg batch size benchmarks for streaming 

In [2]:
# Benchmark
avg_log_size=0.478 # in kilobytes

Provide the filepath of your FIL model

In [3]:
# FIL
model_file=IOT_XGBOOST_MODEL

Kafka parameters

In [4]:
# Kafka
broker="localhost:9092"
input_topic="input"
output_topic="output"

producer_conf = {
    "bootstrap.servers": broker,
    "session.timeout.ms": 10000,
}

# Dask

Next, create your dask cuda cluster and initialize each dask worker with the FIL model referenced above

In [5]:
cluster = LocalCluster()
client = Client(cluster)

In [6]:
def worker_init():
    # Initialization for each dask worker
    import xgboost as xgb
    worker = dask.distributed.get_worker()
    worker.data["cpu_model"] = xgb.Booster()
    worker.data["cpu_model"].load_model(model_file)
    worker.data["cpu_model"].set_param({"predictor": "cpu_predictor"})

In [7]:
client.run(worker_init)

{'tcp://127.0.0.1:33167': None,
 'tcp://127.0.0.1:33365': None,
 'tcp://127.0.0.1:34235': None,
 'tcp://127.0.0.1:35075': None,
 'tcp://127.0.0.1:36583': None,
 'tcp://127.0.0.1:36705': None,
 'tcp://127.0.0.1:36853': None,
 'tcp://127.0.0.1:39381': None,
 'tcp://127.0.0.1:44477': None,
 'tcp://127.0.0.1:46621': None}

In [8]:
print(client)

<Client: 'tcp://127.0.0.1:42097' processes=10 threads=80, memory=540.94 GB>


# Streamz Pipeline

Update the `max_batch_size` and `poll_interval` parameters as needed to tune your streamz workload to suit your environment

In [9]:
max_batch_size=900000
poll_interval="1s"

In [10]:
# Generate a unique group_id to be able to re-run this demo notebook on the same data loaded to your kafka topic.
j = random.randint(0,10000)
group_id="fil-group-%d" % j

# Kafka consumer configuration
consumer_conf = {
    "bootstrap.servers": broker,
    "group.id": group_id,
    "session.timeout.ms": "60000",
    "enable.partition.eof": "true",
    "auto.offset.reset": "earliest",
}

source = Stream.from_kafka_batched(
        input_topic,
        consumer_conf,
        poll_interval=poll_interval,
        npartitions=1,
        asynchronous=True,
        dask=True,
        max_batch_size=max_batch_size,
)

Next, we define the `predict` function to be used in the streamz pipeline. The predict function will construct a GPU dataframe of the raw log messages from kafka, format the data and then execute a prediction using the FIL model we previously loaded into Dask.

In [11]:
def predict(messages):
    batch_start_time = int(round(time.time()))
    worker = dask.distributed.get_worker()
    df = pd.DataFrame()
    if type(messages) == str:
       df["stream"] = [messages.decode('utf-8')]
    elif type(messages) == list and len(messages) > 0:
       df["stream"] = [msg.decode('utf-8') for msg in messages]
    else:
       print("ERROR: Unknown type encountered in inference")
    df['stream'] = df['stream'].astype(str)
    df_conn = pd.json_normalize(df['stream'].apply(json.loads))
    cpu_preds = pd.DataFrame()
    Dmatrix = xgb.DMatrix(df_conn[["resp_ip_bytes", "resp_pkts", "orig_ip_bytes", "orig_pkts"]])
    cpu_preds['predictions'] = worker.data["cpu_model"].predict(Dmatrix)
    size = len(cpu_preds)
    return (cpu_preds, batch_start_time, size)

The `sink_to_kafka` function writes the output data or FIL predictions to the previously defined kafka topic.

In [12]:
def sink_to_kafka(processed_data):
    producer = ck.Producer(producer_conf)
    json_str = processed_data[0].to_json(orient="records", lines=True)
    json_recs = json_str.split("\n")
    print(json_recs)
    for idx,rec in enumerate(json_recs):
        if idx % 50000 == 0:
            producer.flush()
        producer.produce(output_topic, rec)
    producer.flush()
    return processed_data

Below we define our streamz pipeline. This pipeline is also designed to capture benchmark data for reading and processing FIL predictions. 

In [13]:
output = source.map(predict).map(lambda x: (x[0], x[1], int(round(time.time())), x[2])).map(sink_to_kafka).gather().sink_to_list()

Next we start the streamz pipeline. View the progress on your dask dashboard http://localhost:8787

In [14]:
source.start()

This function calculates the benchmark. With each batch of data processed we have recorded the start and stop times that we can then use to calculate the total time difference. Throughput and avg batch size are estimates based on the average log size previously defined.

In [15]:
def calc_benchmark(results, size_per_log):
    t1 = int(round(time.time() * 1000))
    t2 = 0
    size = 0.0
    batch_count = 0
    cnt = 0
    # Find min and max time while keeping track of batch count and size
    for result in results:
        (ts1, ts2, result_size) = (result[1], result[2], result[3])
        cnt += result_size
        if ts1 == 0 or ts2 == 0:
            continue
        batch_count = batch_count + 1
        t1 = min(t1, ts1)
        t2 = max(t2, ts2)
        size += result_size * size_per_log
    time_diff = t2 - t1
    throughput_mbps = size / (1024.0 * time_diff) if time_diff > 0 else 0
    avg_batch_size = size / (1024.0 * batch_count) if batch_count > 0 else 0
    return (time_diff, throughput_mbps, avg_batch_size, cnt)

Please wait a few moments for all logs to be processed before calculating benchmarks  
View the progress on the dask dashboard http://localhost:8787

In [20]:
benchmark = calc_benchmark(output, avg_log_size)
print("max batch size:", max_batch_size)
print("poll interval:", poll_interval)
print("time (s):", benchmark[0])
print("throughput (mb/s):", benchmark[1])
print("avg batch size (mb):", benchmark[2])
print("num records:", benchmark[3])

max batch size: 900000
poll interval: 1s
time (s): 653
throughput (mb/s): 35.74253378038859
avg batch size (mb): 416.7834742606027
num records: 50000066


View the GPU version view this notebook - [FIL Streamz GPU](./FIL_streamz_pipeline_GPU.ipynb)